# Canopy Height Model (CHM) Generation - CyVerse/Local Version

Generate Canopy Height Models from USGS 3D Elevation Program (3DEP) lidar data.

**Modified from:** [OpenTopography OT_3DEP_Workflows](https://github.com/OpenTopography/OT_3DEP_Workflows)

## Key Features

- **Adaptive Resolution:** 0.5m for <12 points/m², 0.333m for >=12 points/m²
- **Direct CHM Calculation:** Simple DSM - DTM subtraction (no smoothing/IDW)
- **CyVerse Compatible:** Designed for CyVerse VICE JupyterLab environment
- **Batch Ready:** Modular design for multi-site processing

## Requirements

```bash
conda activate 3dep
```

## 1. Environment Setup

In [2]:
# Environment validation
import sys

REQUIRED_PACKAGES = ['pdal', 'geopandas', 'rioxarray', 'pyproj', 'shapely', 'numpy']
missing = []

for pkg in REQUIRED_PACKAGES:
    try:
        __import__(pkg)
    except ImportError:
        missing.append(pkg)

if missing:
    raise ImportError(
        f"Missing packages: {missing}\n"
        f"Please activate the 3dep conda environment:\n"
        f"  conda activate 3dep"
    )

print("Environment validated successfully!")

Environment validated successfully!


In [3]:
# Import libraries
import copy
import json
import math
import os
from pathlib import Path
from datetime import datetime

import geopandas as gpd
import ipyleaflet
import ipywidgets as widgets
import numpy as np
from osgeo import gdal
import pdal
import pyproj
import requests
import rioxarray as rio
from rasterio.enums import Resampling
from shapely.geometry import shape, Point, Polygon, box
from shapely.ops import transform
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

print(f"Libraries loaded at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Libraries loaded at 2025-12-18 22:46:26


## 2. Configuration

In [6]:
# Output configuration
# For CyVerse: /iplant/home/<username>/3dep/
# For local: ./outputs/3dep/

OUTPUT_BASE = Path("/data-store/iplant/home/tswetnam/fractal-notebooks/docs/notebooks/3dep/outputs/")
OUTPUT_CHM = OUTPUT_BASE / "chm"
OUTPUT_DTM = OUTPUT_BASE / "dtm"
OUTPUT_DSM = OUTPUT_BASE / "dsm"

# Create output directories
for d in [OUTPUT_CHM, OUTPUT_DTM, OUTPUT_DSM]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Output directories created under: {OUTPUT_BASE.absolute()}")

Output directories created under: /data-store/iplant/home/tswetnam/fractal-notebooks/docs/notebooks/3dep/outputs


In [7]:
# Resolution configuration
# Resolution is determined by point density:
#   - >= 12 points/m² -> 0.333m resolution
#   - <  12 points/m² -> 0.5m resolution

DENSITY_THRESHOLD = 12  # points per square meter
RESOLUTION_HIGH = 0.333  # for high density (>= threshold)
RESOLUTION_STANDARD = 0.5  # for standard density (< threshold)

def get_resolution(point_density_per_m2):
    """Select output resolution based on point density.
    
    Args:
        point_density_per_m2: Estimated point density in points/m²
        
    Returns:
        Resolution in meters (0.333 or 0.5)
    """
    if point_density_per_m2 >= DENSITY_THRESHOLD:
        return RESOLUTION_HIGH
    return RESOLUTION_STANDARD

print(f"Resolution rules:")
print(f"  >= {DENSITY_THRESHOLD} pts/m² -> {RESOLUTION_HIGH}m")
print(f"  <  {DENSITY_THRESHOLD} pts/m² -> {RESOLUTION_STANDARD}m")

Resolution rules:
  >= 12 pts/m² -> 0.333m
  <  12 pts/m² -> 0.5m


## 3. Utility Functions

In [ ]:
def proj_to_3857(poly, orig_crs):
    """Reproject polygon to Web Mercator (EPSG:3857).
    
    Args:
        poly: Shapely polygon
        orig_crs: Original CRS string
        
    Returns:
        Tuple of (polygon in EPSG:4326, polygon in EPSG:3857)
    """
    wgs84 = pyproj.CRS("EPSG:4326")
    web_mercator = pyproj.CRS("EPSG:3857")
    project_gcs = pyproj.Transformer.from_crs(orig_crs, wgs84, always_xy=True).transform
    project_wm = pyproj.Transformer.from_crs(orig_crs, web_mercator, always_xy=True).transform
    return transform(project_gcs, poly), transform(project_wm, poly)


def gcs_to_proj(poly):
    """Reproject from EPSG:4326 to EPSG:3857."""
    wgs84 = pyproj.CRS("EPSG:4326")
    web_mercator = pyproj.CRS("EPSG:3857")
    project = pyproj.Transformer.from_crs(wgs84, web_mercator, always_xy=True).transform
    return transform(project, poly)


def bbox_to_polygon(bbox, crs="EPSG:4326"):
    """Convert bounding box to Shapely polygon.
    
    Args:
        bbox: List of [west, south, east, north]
        crs: Coordinate reference system
        
    Returns:
        Tuple of (polygon in EPSG:4326, polygon in EPSG:3857)
    """
    west, south, east, north = bbox
    poly = box(west, south, east, north)
    
    if crs == "EPSG:4326":
        poly_3857 = gcs_to_proj(poly)
        return poly, poly_3857
    else:
        return proj_to_3857(poly, crs)


def estimate_point_density(aoi_area_m2, total_dataset_points, dataset_area_m2):
    """Estimate point density for an AOI.
    
    Args:
        aoi_area_m2: Area of interest in square meters
        total_dataset_points: Total points in the 3DEP dataset
        dataset_area_m2: Total area of the 3DEP dataset in square meters
        
    Returns:
        Estimated points per square meter
    """
    dataset_density = total_dataset_points / dataset_area_m2
    return dataset_density

In [ ]:
def build_pdal_pipeline(
    extent_epsg3857,
    usgs_3dep_dataset_names,
    pc_resolution,
    filter_noise=True,
    out_crs=3857
):
    """Build PDAL pipeline for point cloud access.
    
    Args:
        extent_epsg3857: AOI polygon in EPSG:3857 (WKT string)
        usgs_3dep_dataset_names: List of 3DEP dataset names
        pc_resolution: Point cloud resolution parameter
        filter_noise: Remove noise classes (7 and 18)
        out_crs: Output CRS EPSG code
        
    Returns:
        PDAL pipeline dictionary
    """
    readers = []
    for name in usgs_3dep_dataset_names:
        url = f"https://s3-us-west-2.amazonaws.com/usgs-lidar-public/{name}/ept.json"
        reader = {
            "type": "readers.ept",
            "filename": url,
            "polygon": str(extent_epsg3857),
            "requests": 3,
            "resolution": pc_resolution
        }
        readers.append(reader)
    
    pipeline = {"pipeline": readers}
    
    if filter_noise:
        # Remove Class 7 (low noise) and Class 18 (high noise)
        pipeline['pipeline'].append({
            "type": "filters.range",
            "limits": "Classification![7:7]"
        })
        pipeline['pipeline'].append({
            "type": "filters.range",
            "limits": "Classification![18:18]"
        })
    
    # Reproject to output CRS
    pipeline['pipeline'].append({
        "type": "filters.reprojection",
        "out_srs": f"EPSG:{out_crs}"
    })
    
    return pipeline


def make_dem_pipeline(
    extent_epsg3857,
    usgs_3dep_dataset_names,
    pc_resolution,
    dem_resolution,
    dem_type,  # 'dsm' or 'dtm'
    dem_out_path,
    filter_noise=True,
    out_crs=3857
):
    """Build PDAL pipeline for DEM generation.
    
    IMPORTANT: Uses 'max' for DSM and 'min' for DTM - NO IDW smoothing.
    
    Args:
        extent_epsg3857: AOI polygon in EPSG:3857 (WKT string)
        usgs_3dep_dataset_names: List of 3DEP dataset names
        pc_resolution: Point cloud resolution parameter
        dem_resolution: Output DEM resolution in meters
        dem_type: 'dsm' (all points) or 'dtm' (ground only)
        dem_out_path: Output file path
        filter_noise: Remove noise classes
        out_crs: Output CRS EPSG code
        
    Returns:
        PDAL pipeline dictionary
    """
    pipeline = build_pdal_pipeline(
        extent_epsg3857, usgs_3dep_dataset_names, pc_resolution,
        filter_noise=filter_noise, out_crs=out_crs
    )
    
    if dem_type == 'dtm':
        # Filter to ground points only (Class 2)
        pipeline['pipeline'].append({
            "type": "filters.range",
            "limits": "Classification[2:2]"
        })
        # Use MIN for DTM (lowest point = ground)
        grid_method = "min"
    else:
        # Use MAX for DSM (highest point = canopy/surface)
        grid_method = "max"
    
    # Add GDAL writer - NO IDW, direct gridding only
    pipeline['pipeline'].append({
        "type": "writers.gdal",
        "filename": str(dem_out_path),
        "gdaldriver": "GTiff",
        "nodata": -9999,
        "output_type": grid_method,  # 'max' for DSM, 'min' for DTM
        "resolution": float(dem_resolution),
        "gdalopts": "COMPRESS=LZW,TILED=YES,BLOCKXSIZE=256,BLOCKYSIZE=256"
    })
    
    return pipeline

## 4. Load 3DEP Dataset Boundaries

In [ ]:
print("Loading 3DEP dataset boundaries...")

# Fetch 3DEP boundaries from Hobu Inc repository
url = 'https://raw.githubusercontent.com/hobuinc/usgs-lidar/master/boundaries/resources.geojson'
r = requests.get(url)

# Save locally for reference
boundaries_file = OUTPUT_BASE / 'resources.geojson'
with open(boundaries_file, 'w') as f:
    f.write(r.content.decode("utf-8"))

# Load as GeoDataFrame
df_3dep = gpd.read_file(boundaries_file)

# Project geometries to EPSG:3857
df_3dep['geometry_3857'] = df_3dep['geometry'].apply(gcs_to_proj)

print(f"Loaded {len(df_3dep)} 3DEP datasets")
print(f"Total points available: {df_3dep['count'].sum():,.0f}")

## 5. Define Area of Interest (AOI)

Two options:
1. **Define by GeoJSON** - Provide polygon coordinates directly (supports multiple polygons)
2. **Interactive map** - Draw polygon on map

Current AOI: **Monument Canyon Research Natural Area, NM**

In [ ]:
# Define AOI using GeoJSON
# Monument Canyon Research Natural Area, NM

AOI_GEOJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [-106.64270398641813, 35.81386115600991],
            [-106.64270398641813, 35.79252004986472],
            [-106.6172480729282, 35.79252004986472],
            [-106.6172480729282, 35.81386115600991],
            [-106.64270398641813, 35.81386115600991]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

SITE_NAME = "monument_canyon_rna"

# Parse GeoJSON and create AOI
from shapely.geometry import shape

feature = AOI_GEOJSON['features'][0]
AOI_GCS = shape(feature['geometry'])
AOI_EPSG3857 = gcs_to_proj(AOI_GCS)

# Calculate bounding box for reference
AOI_BBOX = list(AOI_GCS.bounds)  # [west, south, east, north]

print(f"Site: {SITE_NAME}")
print(f"Bounding box: {AOI_BBOX}")
print(f"Area: {AOI_EPSG3857.area / 1e6:.4f} km²")

In [ ]:
# Option 2: Interactive map (uncomment to use)
# Note: After drawing, run the cell below to extract the AOI

"""
user_AOI = []

def handle_draw(target, action, geo_json):
    geom = dict(geo_json['geometry'])
    user_poly = shape(geom)
    user_poly_proj3857 = gcs_to_proj(user_poly)
    print(f'AOI bounds: {user_poly_proj3857.bounds}')
    user_AOI.append((user_poly, user_poly_proj3857))

# Create map
m = ipyleaflet.Map(
    basemap=ipyleaflet.basemaps.Esri.WorldTopoMap,
    center=(37, -100),
    zoom=4
)

# Add draw control
dc = ipyleaflet.DrawControl(
    polygon={"shapeOptions": {"color": "blue"}},
    rectangle={"shapeOptions": {"color": "blue"}},
    circlemarker={},
    polyline={}
)
dc.on_draw(handle_draw)
m.add_control(dc)

display(m)
"""
print("Interactive map disabled. Using bounding box from previous cell.")

## 6. Find Intersecting 3DEP Datasets

In [ ]:
# Find 3DEP datasets that intersect with our AOI
intersecting = []

for idx, row in df_3dep.iterrows():
    if row['geometry_3857'].intersects(AOI_EPSG3857):
        intersecting.append({
            'name': row['name'],
            'url': row['url'],
            'count': row['count'],
            'geometry_gcs': row['geometry'],
            'geometry_3857': row['geometry_3857']
        })

if not intersecting:
    raise ValueError("No 3DEP datasets found for this AOI. Check coordinates.")

print(f"Found {len(intersecting)} intersecting 3DEP dataset(s):")
for ds in intersecting:
    print(f"  - {ds['name']}: {ds['count']:,} points")

# Extract dataset names for PDAL
dataset_names = [ds['name'] for ds in intersecting]

## 7. Estimate Point Density and Select Resolution

In [ ]:
# Estimate point density for our AOI
total_points_estimate = 0
total_dataset_area = 0

for ds in intersecting:
    # Ratio of AOI area to dataset area
    ratio = AOI_EPSG3857.area / ds['geometry_3857'].area
    points_estimate = ratio * ds['count']
    total_points_estimate += points_estimate
    total_dataset_area += ds['geometry_3857'].area

# Calculate density
aoi_area_m2 = AOI_EPSG3857.area
estimated_density = total_points_estimate / aoi_area_m2

# Select resolution based on density
selected_resolution = get_resolution(estimated_density)

print(f"AOI Area: {aoi_area_m2 / 1e6:.4f} km² ({aoi_area_m2:,.0f} m²)")
print(f"Estimated points: {total_points_estimate:,.0f}")
print(f"Estimated density: {estimated_density:.1f} points/m²")
print(f"")
print(f"Selected resolution: {selected_resolution}m")
print(f"  (Threshold: {DENSITY_THRESHOLD} pts/m² -> "
      f"{'HIGH' if estimated_density >= DENSITY_THRESHOLD else 'STANDARD'} resolution)")

## 8. Generate Digital Surface Model (DSM)

In [ ]:
# DSM output path
dsm_path = OUTPUT_DSM / f"{SITE_NAME}_dsm.tif"

print(f"Generating DSM at {selected_resolution}m resolution...")
print(f"Output: {dsm_path}")
print(f"Method: MAX (highest point per cell - NO IDW)")

# Build pipeline
dsm_pipeline_dict = make_dem_pipeline(
    extent_epsg3857=AOI_EPSG3857.wkt,
    usgs_3dep_dataset_names=dataset_names,
    pc_resolution=1.0,  # Full resolution point cloud
    dem_resolution=selected_resolution,
    dem_type='dsm',
    dem_out_path=dsm_path,
    filter_noise=True,
    out_crs=3857
)

# Execute
dsm_pipeline = pdal.Pipeline(json.dumps(dsm_pipeline_dict))

start_time = datetime.now()
dsm_pipeline.execute_streaming(chunk_size=1000000)
elapsed = (datetime.now() - start_time).total_seconds()

print(f"DSM complete in {elapsed:.1f} seconds")

## 9. Generate Digital Terrain Model (DTM)

In [ ]:
# DTM output path
dtm_path = OUTPUT_DTM / f"{SITE_NAME}_dtm.tif"

print(f"Generating DTM at {selected_resolution}m resolution...")
print(f"Output: {dtm_path}")
print(f"Method: MIN (lowest ground point per cell - NO IDW)")

# Build pipeline
dtm_pipeline_dict = make_dem_pipeline(
    extent_epsg3857=AOI_EPSG3857.wkt,
    usgs_3dep_dataset_names=dataset_names,
    pc_resolution=1.0,  # Full resolution point cloud
    dem_resolution=selected_resolution,
    dem_type='dtm',
    dem_out_path=dtm_path,
    filter_noise=True,
    out_crs=3857
)

# Execute
dtm_pipeline = pdal.Pipeline(json.dumps(dtm_pipeline_dict))

start_time = datetime.now()
dtm_pipeline.execute_streaming(chunk_size=1000000)
elapsed = (datetime.now() - start_time).total_seconds()

print(f"DTM complete in {elapsed:.1f} seconds")

## 10. Calculate Canopy Height Model (CHM)

**CHM = DSM - DTM**

Simple direct subtraction with no smoothing or interpolation.

In [ ]:
# Load DSM and DTM
print("Loading DSM and DTM...")
dsm = rio.open_rasterio(dsm_path, masked=True)
dtm = rio.open_rasterio(dtm_path, masked=True)

print(f"DSM shape: {dsm.shape}")
print(f"DTM shape: {dtm.shape}")

In [ ]:
# Align rasters if needed
if dsm.shape != dtm.shape:
    print("Aligning rasters...")
    if dsm.shape > dtm.shape:
        dsm = dsm.rio.reproject_match(dtm)
    else:
        dtm = dtm.rio.reproject_match(dsm)
    print(f"Aligned shapes: DSM={dsm.shape}, DTM={dtm.shape}")

# Ensure coordinates match exactly
dsm = dsm.assign_coords({"x": dtm.x, "y": dtm.y})

In [ ]:
# Calculate CHM: Direct subtraction only
print("Calculating CHM = DSM - DTM (direct subtraction, no smoothing)...")

chm = dsm - dtm
chm = chm.compute()

# Set NoData value
chm.rio.set_nodata(dtm.rio.nodata, inplace=True)

# Save CHM
chm_path = OUTPUT_CHM / f"{SITE_NAME}_chm.tif"
chm.rio.to_raster(chm_path)

print(f"CHM saved to: {chm_path}")

## 11. Calculate Statistics and Generate Metadata

In [ ]:
# Calculate CHM statistics
chm_data = chm.values.flatten()
chm_valid = chm_data[~np.isnan(chm_data)]
chm_valid = chm_valid[chm_valid != chm.rio.nodata]

# Filter to reasonable height values (0-150m)
chm_heights = chm_valid[(chm_valid >= 0) & (chm_valid <= 150)]

stats = {
    'site_name': SITE_NAME,
    'bbox': AOI_BBOX,
    'area_km2': aoi_area_m2 / 1e6,
    'resolution_m': selected_resolution,
    'point_density_est': estimated_density,
    'datasets_used': dataset_names,
    'processing': {
        'dsm_method': 'max',
        'dtm_method': 'min',
        'smoothing': 'none',
        'interpolation': 'none'
    },
    'chm_statistics': {
        'min_height_m': float(np.min(chm_heights)) if len(chm_heights) > 0 else None,
        'max_height_m': float(np.max(chm_heights)) if len(chm_heights) > 0 else None,
        'mean_height_m': float(np.mean(chm_heights)) if len(chm_heights) > 0 else None,
        'median_height_m': float(np.median(chm_heights)) if len(chm_heights) > 0 else None,
        'std_height_m': float(np.std(chm_heights)) if len(chm_heights) > 0 else None,
        'valid_pixels': int(len(chm_heights)),
        'total_pixels': int(len(chm_data))
    },
    'generated_at': datetime.now().isoformat(),
    'source': 'USGS 3DEP via OpenTopography'
}

# Print summary
print("CHM Statistics:")
print(f"  Min height:    {stats['chm_statistics']['min_height_m']:.2f} m")
print(f"  Max height:    {stats['chm_statistics']['max_height_m']:.2f} m")
print(f"  Mean height:   {stats['chm_statistics']['mean_height_m']:.2f} m")
print(f"  Median height: {stats['chm_statistics']['median_height_m']:.2f} m")
print(f"  Std deviation: {stats['chm_statistics']['std_height_m']:.2f} m")

In [ ]:
# Save metadata
metadata_path = OUTPUT_CHM / f"{SITE_NAME}_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(stats, f, indent=2)

print(f"Metadata saved to: {metadata_path}")

## 12. Visualize CHM

In [ ]:
# Downsample for visualization if needed
def downsample_for_plot(raster, max_dim=1000):
    """Downsample raster for efficient plotting."""
    if max(raster.shape) > max_dim:
        scale = max_dim / max(raster.shape[1], raster.shape[2])
        new_width = int(raster.rio.width * scale)
        new_height = int(raster.rio.height * scale)
        return raster.rio.reproject(
            raster.rio.crs,
            shape=(new_height, new_width),
            resampling=Resampling.bilinear
        )
    return raster

chm_plot = downsample_for_plot(chm.squeeze())

In [ ]:
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# CHM map
ax1 = axes[0]
im = chm_plot.plot(ax=ax1, cmap='Greens', robust=True, add_colorbar=False)
ax1.set_title(f"Canopy Height Model: {SITE_NAME}")
ax1.set_xlabel("Easting (m)")
ax1.set_ylabel("Northing (m)")
ax1.ticklabel_format(style='plain')
ax1.set_aspect('equal')
plt.colorbar(im, ax=ax1, label='Height (m)')

# Height histogram
ax2 = axes[1]
ax2.hist(chm_heights, bins=50, color='green', alpha=0.7, edgecolor='darkgreen')
ax2.axvline(stats['chm_statistics']['mean_height_m'], color='red', 
            linestyle='--', label=f"Mean: {stats['chm_statistics']['mean_height_m']:.1f}m")
ax2.axvline(stats['chm_statistics']['median_height_m'], color='orange', 
            linestyle='--', label=f"Median: {stats['chm_statistics']['median_height_m']:.1f}m")
ax2.set_title("Height Distribution")
ax2.set_xlabel("Canopy Height (m)")
ax2.set_ylabel("Pixel Count")
ax2.legend()

plt.tight_layout()

# Save figure
preview_path = OUTPUT_CHM / f"{SITE_NAME}_preview.png"
plt.savefig(preview_path, dpi=150, bbox_inches='tight')
print(f"Preview saved to: {preview_path}")

plt.show()

## 13. Cleanup

In [ ]:
# Close raster handles
dsm.close()
dtm.close()
chm.close()

del dsm, dtm, chm

print("\n" + "="*50)
print("CHM Generation Complete!")
print("="*50)
print(f"\nOutputs in: {OUTPUT_BASE.absolute()}")
print(f"  - CHM: {chm_path.name}")
print(f"  - DSM: {dsm_path.name}")
print(f"  - DTM: {dtm_path.name}")
print(f"  - Metadata: {metadata_path.name}")
print(f"  - Preview: {preview_path.name}")

---

## References

- Original notebook: [OpenTopography OT_3DEP_Workflows](https://github.com/OpenTopography/OT_3DEP_Workflows)
- 3DEP Program: https://www.usgs.gov/3d-elevation-program
- PDAL Documentation: https://pdal.io/